In [ ]:
pip install diffusers transformers torch pandas requests pillow


In [ ]:
from diffusers import StableDiffusionPipeline
import pandas as pd
import torch
from PIL import Image
import os


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cuda") if torch.cuda.is_available() else pipe.to("cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def generate_image(description, output_image_path):
    """
    Generate an image using Stable Diffusion based on the disease and symptom description.
    """
    image = pipe(description).images[0]
    image.save(output_image_path)

In [ ]:
def update_csv_with_images_in_batches(csv_file, output_image_column, image_folder, batch_size=10):
    """
    Update the CSV file by generating images in batches for each disease and its symptoms,
    and adding the image paths to a new column.
    """
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Create the folder if it doesn't exist
    if not os.path.exists(image_folder):
        os.makedirs(image_folder)

    # Iterate through each row in the CSV in batches
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_df = df[start_idx:end_idx]

        # Process each row in the current batch
        for index, row in batch_df.iterrows():
            # Split the disease and symptoms
            disease = row['Disease']
            symptoms = [row['Symptom_1'],row['Symptom_2'],row['Symptom_3']]

            # Store all image paths for this row
            image_paths = []

            # Generate an image for each symptom
            for i, symptom in enumerate(symptoms):
                prompt = f"Symptom of {disease}: {symptom}"

                # Define the image file path
                image_file_name = f"{disease.replace(' ', '_').lower()}_symptom_{i+1}_{index}.png"
                image_path = os.path.join(image_folder, image_file_name)

                # Generate and save the image
                generate_image(prompt, image_path)

                # Add the image path to the list
                image_paths.append(image_path)

            # Update the dataframe with the joined image paths
            df.at[index, output_image_column] = '; '.join(image_paths)

        # Save progress after each batch to avoid losing data in case of interruptions
        df.to_csv(csv_file, index=False)
        print(f"Batch {start_idx // batch_size + 1} saved successfully.")

    print("CSV updated successfully with image paths.")



In [ ]:
from google.colab import files
uploaded = files.upload()


Saving diseases_and_symptoms (2).csv to diseases_and_symptoms (2).csv


In [ ]:
!ls

'diseases_and_symptoms (2).csv'   sample_data


In [ ]:
csv_file_path = 'diseases_and_symptoms (2).csv'
df_updated = pd.read_csv(csv_file_path)
# Display the DataFrame
df_updated.head()

,Disease,Symptom_1,Symptom_2,Symptom_3
0,Alcoholic hepatitis,vomiting,yellowish_skin,abdominal_pain
1,Paralysis (brain hemorrhage),vomiting,headache,weakness_of_one_body_side
2,Gastroenteritis,vomiting,sunken_eyes,dehydration
3,Tuberculosis,chills,vomiting,fatigue
4,Drug Reaction,itching,skin_rash,stomach_pain


In [ ]:
output_image_column = 'Image_Paths'      # The new column to store the image paths (for multiple images)
image_output_folder = 'disease_images'  # Folder to save images
update_csv_with_images_in_batches(csv_file_path, output_image_column, image_output_folder)

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


...

Batch 4 saved successfully.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Batch 5 saved successfully.
CSV updated successfully with image paths.


In [ ]:
!ls

 disease_images  'diseases_and_symptoms (2).csv'   sample_data


In [ ]:
# Compress the disease_images directory into a zip file
!zip -r disease_images.zip disease_images
from google.colab import files
# Download the zip file
files.download('disease_images.zip')


updating: disease_images/ (stored 0%)
updating: disease_images/drug_reaction_symptom_1_4.png (deflated 0%)
updating: disease_images/allergy_symptom_3_23.png (deflated 0%)
updating: disease_images/peptic_ulcer_diseae_symptom_1_21.png (deflated 0%)
updating: disease_images/jaundice_symptom_2_38.png (deflated 0%)
updating: disease_images/heart_attack_symptom_3_8.png (deflated 0%)
updating: disease_images/arthritis_symptom_3_17.png (deflated 0%)
updating: disease_images/diabetes__symptom_1_37.png (deflated 0%)
updating: disease_images/jaundice_symptom_1_38.png (deflated 0%)
updating: disease_images/common_cold_symptom_1_10.png (deflated 0%)
updating: disease_images/allergy_symptom_2_23.png (deflated 0%)
updating: disease_images/pneumonia_symptom_1_20.png (deflated 0%)
updating: disease_images/fungal_infection_symptom_2_19.png (deflated 0%)
updating: disease_images/chicken_pox_symptom_2_16.png (deflated 0%)
updating: disease_images/varicose_veins_symptom_3_24.png (deflated 0%)
updating: dis

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Download the CSV to computer
from google.colab import files
files.download('diseases_and_symptoms (2).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
updated_csv_path = 'diseases_and_symptoms (2).csv'  # Path to your updated CSV file
df_updated = pd.read_csv(updated_csv_path)

# Display the DataFrame
df_updated.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Image_Paths
0,Alcoholic hepatitis,vomiting,yellowish_skin,abdominal_pain,disease_images/alcoholic_hepatitis_symptom_1_0...
1,Paralysis (brain hemorrhage),vomiting,headache,weakness_of_one_body_side,disease_images/paralysis_(brain_hemorrhage)_sy...
2,Gastroenteritis,vomiting,sunken_eyes,dehydration,disease_images/gastroenteritis_symptom_1_2.png...
3,Tuberculosis,chills,vomiting,fatigue,disease_images/tuberculosis_symptom_1_3.png; d...
4,Drug Reaction,itching,skin_rash,stomach_pain,disease_images/drug_reaction_symptom_1_4.png; ...
